# Event Study Framework: Computing CAPM Adjusted Returns

In this notebook I'll present a framework to perform an event study using CAPM adjusted returns: 1 event, several firms

1. Extractions   

    1.1 FF factors   
    1.2 Stock Returns    
    
    
2. Computing the abnormal and cumulative abnormal returns

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model #running linear regressions
import statsmodels.api as sm #to run regressions
import pandas_datareader as web #to download prices from Yahoo Finance
import urllib.request #to extract FF factors
import zipfile #to unzip files
import datetime #to work with the dates

Let's start by defining the event window. In this section, I'll consider one event only, that may affect several firms.

If you want to study several events (such as M&A announcements), you may skip to section 2

In [2]:
event_date = datetime.datetime.strptime("2020-02-03",'%Y-%m-%d') #choose the date of your event

### 1. Extractions

#### 1.1 Extracting FF factors

In [3]:
def get_fama_french():
    # Web url
    ff_url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip"
    
    # Download the file and save it
    # We will name it fama_french.zip file
    
    urllib.request.urlretrieve(ff_url,'fama_french.zip')
    zip_file = zipfile.ZipFile('fama_french.zip', 'r')
    
    # Next we extact the file data
    
    zip_file.extractall()
    
    # Make sure you close the file after extraction
    
    zip_file.close()
    
    # Now open the CSV file
    
    ff_factors = pd.read_csv('F-F_Research_Data_Factors_daily.csv', skiprows = 3, index_col = 0)
    # We want to find out the row with NULL value
    # We will skip these rows
    
    ff_row = ff_factors.isnull().any(1).nonzero()[0][0]
    
    # Read the csv file again with skipped rows
    ff_factors = pd.read_csv('F-F_Research_Data_Factors_daily.csv', skiprows = 3, nrows = ff_row, index_col = 0)
    
    # Format the date index
    ff_factors.index = pd.to_datetime(ff_factors.index, format= '%Y%m%d')
    
    
    # Convert from percent to decimal
    ff_factors = ff_factors.apply(lambda x: x/ 100)
    return ff_factors

In [4]:
#Let's see if it worked:
get_fama_french().head()

,Mkt-RF,SMB,HML,RF
1926-07-01,0.0010,-0.0024,-0.0028,0.00009
1926-07-02,0.0045,-0.0032,-0.0008,0.00009
1926-07-06,0.0017,0.0027,-0.0035,0.00009
1926-07-07,0.0009,-0.0059,0.0003,0.00009
1926-07-08,0.0021,-0.0036,0.0015,0.00009


In [5]:
#let's save it to a dataframe (we just need the rf and the Mkt-RF for the CAPM)
ff = get_fama_french().loc[:,['Mkt-RF', 'RF']]

#### 1.2 Extracting Stock Returns

In [6]:
#list of firms we want to study:
firms = ['AAPL', 'MSFT', 'GOOGL']

#setting a start date and end date - according to the period you want to study
start_date = '01-01-2019'
end_date = '03-02-2020'

returns= pd.DataFrame()
for firm in firms:
    returns[firm] = web.get_data_yahoo(firm, start_date, end_date)['Adj Close'] #extracting prices

returns = returns.pct_change()[1:] #turning prices into returns

returns.head()

,AAPL,MSFT,GOOGL
Date,,,
2019-01-03,-0.099607,-0.036788,-0.027696
2019-01-04,0.042689,0.046509,0.051294
2019-01-07,-0.002226,0.001275,-0.001994
2019-01-08,0.019063,0.007251,0.008783
2019-01-09,0.016982,0.014300,-0.003427


In [7]:
#Merging FF and stock returns, with the dates of the stock returns
all_returns = pd.merge(pd.DataFrame(returns),ff, how = 'inner', left_index= True, right_index= True)
all_returns.head()

,AAPL,MSFT,GOOGL,Mkt-RF,RF
2019-01-03,-0.099607,-0.036788,-0.027696,-0.0245,0.0001
2019-01-04,0.042689,0.046509,0.051294,0.0355,0.0001
2019-01-07,-0.002226,0.001275,-0.001994,0.0094,0.0001
2019-01-08,0.019063,0.007251,0.008783,0.0101,0.0001
2019-01-09,0.016982,0.014300,-0.003427,0.0056,0.0001


### 2. Abnornal Returns

We may think of abnormal returns as the difference between the expected normal returns (in case the event did not occur) and the actual returns during the event. 

The expected normal returns can be estimated through a historical returns window - defined as the __estimation window__. 

The estimations are made for the period we want to study the impact of the event - it may include days before and after the event - this is defined as the __event window__. 

The picture below shows an example of a framework for event and estimation windows:


![title](https://drive.google.com/uc?id=1BFPOfg7bj-fMolTxqLXwQM0SnzBQnA1B)

We will be running CAPM adjusted returns, hence the estimation window will solely be used to estimate the betas (systematic risk). Once that is done, the abnormal returns are computed according to the formula below:

Abnormal Return = Actual Return - (rf + B*(Mkt - rf))

The rf is the return of the riskless asset, and the Mkt are the market returns - both extracted from Kenneth French website

In [8]:
#Defining the Estimation Window: 
date_first = event_date + datetime.timedelta(days=-252-14) #one year and 14 trading days (20 atual days) before the event
date_last = event_date + datetime.timedelta(days=-14) #14 trading days before the event

#dataframe for abnormal returns:
ar = pd.DataFrame(index = firms)
ar['returns'] = returns.loc[event_date]

#Computing the betas:
betas = list()
for firm in firms:
    betas.append(sm.OLS(endog= all_returns.loc[date_first:date_last, firm], exog = all_returns.loc[date_first:date_last, 'Mkt-RF']).fit().params[0])
ar['betas'] = pd.DataFrame(betas, index= firms)

##### Computing Abnormal Returns on a one-day event window:

In [9]:
#Computing Abnormal returns
#1 day event window
ar['abnormal_1d'] = ar['returns'] - (all_returns.loc[event_date, 'RF'] + ar['betas']*all_returns.loc[event_date,'Mkt-RF'])

In [10]:
ar

,returns,betas,abnormal_1d
AAPL,-0.002746,1.485370,-0.015283
MSFT,0.024379,1.227141,0.014011
GOOGL,0.034772,1.218885,0.024473


##### Computing cumulative abnormal returns   
Ex: 5 days event window: -2 and +2 days from the date of the event

In [11]:
#defining the days: -2 and +2 on the event day:
minus_2 = event_date + datetime.timedelta(days = -2)
plus_2 = event_date + datetime.timedelta(days = +2)

five_days_returns = list() #creating list to append the sum of the returns on the 5 days windows, on each firm
for firm in firms:
    five_days_returns.append(sum(all_returns.loc[minus_2:plus_2,firm]))
ar['returns_5d'] = pd.DataFrame(five_days_returns, index = firms) #inserting them in the dataframe

#computing the five days cumulative abnormal returns:
ar['abnormal_5d'] = ar['returns_5d'] - (sum(all_returns.loc[minus_2:plus_2,'RF']) + \
ar['betas']*sum(all_returns.loc[minus_2:plus_2, 'Mkt-RF']))

In [12]:
ar

,returns,betas,abnormal_1d,returns_5d,abnormal_5d
AAPL,-0.002746,1.485370,-0.015283,0.038422,-0.011964
MSFT,0.024379,1.227141,0.014011,0.056074,0.014417
GOOGL,0.034772,1.218885,0.024473,0.010130,-0.031248


So, you have learned how to calculate (cumulative) abnormal returns for one specific event on several firms. Now you are ready to perform an event study, which you should regress these abnormal returns against some variables you want to study.  

If you are interested in seeing some examples of event studies, you can start with this interesting [paper](https://www.hks.harvard.edu/sites/default/files/centers/mrcbg/files/Zeckhauser_final_2017-01.pdf) from AF Wagner, RJ Zeckhauser and A Ziegler, who study the impact of Donald Trump's election on tax and international trade, looking into stock market reactions